In [1]:
#Importing the data
import pandas as pd
import numpy as np
import pandas as pd
from surprise import Reader, Dataset, KNNBasic, accuracy, PredictionImpossible
from surprise.model_selection import cross_validate
from surprise.model_selection import train_test_split
from surprise.accuracy import rmse
from collections import defaultdict
#Importing the data
data = pd.read_csv('/home/bbruno/all_here/python course/vinnie/data/cleaned_data/df_oversamling.csv')
data.head()

,userId,wine,rate
0,ecc462e1-5041-43ca-94c3-c2bfc9b3215e,Merlot le vigne di zamo,1
1,ecc462e1-5041-43ca-94c3-c2bfc9b3215e,le vigne di zamo refosco,1
2,bf82194f-6108-4524-9cef-69c0231ac74b,Nerello mascalese,1
3,e5b0e77a-ed45-4436-a771-6e01c5782973,Campofiorin Rosso Veronese,1
4,ecc462e1-5041-43ca-94c3-c2bfc9b3215e,le vigne di zamo refosco,1


In [2]:
class Knn (KNNBasic):
    def __init__(self, sim_options={}, bsl_options={}):
        KNNBasic.__init__(self, sim_options=sim_options, bsl_options=bsl_options)
    
    def create_reader(self, data):
        reader = Reader(rating_scale=(1, 5))
        self.data = Dataset.load_from_df(data[['userId', 'wine', 'rate']], reader)
    # option 1
    ########################################
    # for cross validation we have two functions, cross_validate and fit
    def cross_validate(self, measures=['RMSE'], cv=3, verbose=False):
        results = cross_validate(self, self.data, measures=measures, cv=cv, verbose=verbose)
        for measure in measures:
            print(f'{measure}: {results["test_" + measure.lower()].mean()}')
        return results
    
    def fit(self, trainset):
        predictions = KNNBasic.fit(self, trainset).test(trainset.build_testset())
        self.sim = self.compute_similarities()
        self.bu, self.bi = self.compute_baselines()
        return predictions
    ########################################
    # option 2
    ########################################
    # # fit funtion that works without cross validation
    # def fit (self):
    #     self.trainset, testset = train_test_split(self.data, test_size=0.2)
    #     predictions = KNNBasic.fit(self, self.trainset).test(testset)
    #     self.sim = self.compute_similarities()
    #     self.bu, self.bi = self.compute_baselines()
    #     return predictions
    ########################################

    def estimated(self, u, i):
        if not (self.trainset.knows_user(u) and self.trainset.knows_item(i)):
            raise PredictionImpossible("User and/or item is unknown.")
        
        # Compute similarities between u and v, where v describes all other
        # users that have also rated item i.
        neighbors = [(v, self.sim[u, v]) for (v, r) in self.trainset.ir[i]]
        # Sort these neighbors by similarity
        neighbors = sorted(neighbors, key=lambda x: x[1], reverse=True)

        print("The 5 nearest neighbors of user", str(u), "are:")
        for v, sim_uv in neighbors[:5]:
            print(f"user {v} with sim {sim_uv:1.15f}")

        # ... Aaaaand return the baseline estimate anyway ;)
        bsl = self.trainset.global_mean + self.bu[u] + self.bi[i]
        return print(f"And the baseline estimate is: {bsl}")
    
    def get_Iu(self, uid):
        """Return the number of items rated by given user
        args:
          uid: the id of the user
        returns:
          the number of items rated by the user
        """
        try:
            return len(self.trainset.ur[self.trainset.to_inner_uid(uid)])
        except ValueError:  # user was not part of the trainset
            return 0

    def get_Ui(self, iid):
        """Return the number of users that have rated given item
        args:
          iid: the raw id of the item
        returns:
          the number of users that have rated the item.
        """
        try:
            return len(self.trainset.ir[self.trainset.to_inner_iid(iid)])
        except ValueError:
            return 0

    def inspect_predictions(self, predictions):
        print(f"uid means the user id and iid means the wine id\n")
        print(f"rui means the actual rating and est means the estimated rating\n")
        print(f"err means the error between the actual and the estimated rating\n")
        print(f"Iu means the number of items rated by given user\n")
        print(f"Ui means the number of users that have rated given item\n")
        # Create a dataframe with the predictions
        df_pred = pd.DataFrame(predictions, columns=['uid', 'iid', 'rui', 'est', 'details'])
        df_pred['Iu'] = df_pred.uid.apply(self.get_Iu)
        df_pred['Ui'] = df_pred.iid.apply(self.get_Ui)
        df_pred['err'] = abs(df_pred.est - df_pred.rui)
        return df_pred
    
    def get_accuracy(self, predictions, k=10, threshold=3.5):
        # Compute RMSE
        accuracy.rmse(predictions, verbose=True)
        
        # Compute precision and recall
        precisions, recalls = self.precision_recall_at_k(predictions, k=k, threshold=threshold)

        # Precision and recall can then be averaged over all users
        precision = sum(prec for prec in precisions.values()) / len(precisions)
        recall = sum(rec for rec in recalls.values()) / len(recalls)
        print(f'Precision: {precision:.2f}\nRecall: {recall:.2f}')

        # Count correct predictions
        correct = 0
        for uid, iid, true_r, est, _ in predictions:
            if round(est) == round(true_r):
                correct += 1

        # Compute accuracy
        accuracy_percentage = correct / len(predictions)
        return accuracy_percentage * 100
    
    @staticmethod 
    def precision_recall_at_k(predictions, k=10, threshold=3.5):
        """Return precision and recall at k metrics for each user"""

        # First map the predictions to each user.
        user_est_true = defaultdict(list)
        for uid, _, true_r, est, _ in predictions:
            user_est_true[uid].append((est, true_r))

        precisions = dict()
        recalls = dict()
        for uid, user_ratings in user_est_true.items():

            # Sort user ratings by estimated value
            user_ratings.sort(key=lambda x: x[0], reverse=True)

            # Number of relevant items
            n_rel = sum((true_r >= threshold) for (_, true_r) in user_ratings)

            # Number of recommended items in top k
            n_rec_k = sum((est >= threshold) for (est, _) in user_ratings[:k])

            # Number of relevant and recommended items in top k
            n_rel_and_rec_k = sum(
                ((true_r >= threshold) and (est >= threshold))
                for (est, true_r) in user_ratings[:k]
            )

            # Precision@K: Proportion of recommended items that are relevant
            # When n_rec_k is 0, Precision is undefined. We here set it to 0.

            precisions[uid] = n_rel_and_rec_k / n_rec_k if n_rec_k != 0 else 0

            # Recall@K: Proportion of relevant items that are recommended
            # When n_rel is 0, Recall is undefined. We here set it to 0.

            recalls[uid] = n_rel_and_rec_k / n_rel if n_rel != 0 else 0

        return precisions, recalls
    
    # Mi function acc that works perfectly was modifyed to add precision and recall, for that reason this is commented
    # def get_accuracy(self, predictions):
    #     # # Compute RMSE
    #     # predictions = KNNBasic.test(self, testset)
    #     accuracy.rmse(predictions, verbose=True)
    #     # Count correct predictions
    #     correct = 0
    #     for uid, iid, true_r, est, _ in predictions:
    #         if round(est) == round(true_r):
    #             correct += 1

    #     # Compute accuracy
    #     accuracy_percentage = correct / len(predictions)
    #     return accuracy_percentage * 100

Here's a brief explanation of each step:

1. `knn.create_reader(data)`: This loads your data into the KNN model.
2. `knn.cross_validate(cv=3)`: This performs cross-validation on your data with 3 folds. It's good to do this before training your model to get an idea of how well it might perform.
3. `trainset = knn.data.build_full_trainset()`: This builds the full trainset from your data.
4. `predictions = knn.fit(trainset)`: This trains your model on the full trainset and generates predictions.
5. `knn.estimated(140, 10)`: This estimates the rating that user 140 would give to item 10.
6. `df_pred = knn.inspect_predictions(predictions)`: This inspects your predictions.
7. `df_pred.head(10)`: This displays the first 10 rows of your predictions.

For your next steps, you might want to:
- Check the accuracy of your model
- Try different values for `cv` to see how it affects your model's performance
- Experiment with different parameters for your KNN model to see if you can improve its performance.

In [3]:
knn = Knn(
    sim_options = {'name': 'pearson_baseline','user_based': True}, 
    bsl_options={'method': 'sgd', 'learning_rate': 0.00005, 'n_epochs':20, 'reg_u': 12 , 'reg_i': 5}
    )

In [4]:
knn.create_reader(data)

In [5]:
knn.cross_validate(cv=3)

Estimating biases using sgd...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
Estimating biases using sgd...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
Estimating biases using sgd...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
RMSE: 0.6274077042043991


{'test_rmse': array([0.65670904, 0.57644928, 0.6490648 ]),
 'fit_time': (0.026646137237548828,
  0.012238740921020508,
  0.020752906799316406),
 'test_time': (0.004259347915649414,
  0.003985404968261719,
  0.004970073699951172)}

In [6]:
trainset = knn.data.build_full_trainset()
predictions = knn.fit(trainset)
# knn.cross_validate(knn.data)

Estimating biases using sgd...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.


In [7]:
# predictions = knn.fit()

In [8]:
knn.estimated(147, 355)
print("If the baseline is {} then the value is a default value".format(knn.trainset.global_mean))

The 5 nearest neighbors of user 147 are:
user 12 with sim 0.000000000000000
And the baseline estimate is: 3.0089795273854896
If the baseline is 3.0 then the value is a default value


In [9]:
df_pred = knn.inspect_predictions(predictions)
best_pred = df_pred.sort_values(by='err')[:10]
worst_pred = df_pred.sort_values(by='err')[-10:]
df_pred.head(10)

uid means the user id and iid means the wine id

rui means the actual rating and est means the estimated rating

err means the error between the actual and the estimated rating

Iu means the number of items rated by given user

Ui means the number of users that have rated given item



,uid,iid,rui,est,details,Iu,Ui,err
0,ecc462e1-5041-43ca-94c3-c2bfc9b3215e,Merlot le vigne di zamo,1.0,1.0,"{'actual_k': 23, 'was_impossible': False}",167,23,0.0
1,ecc462e1-5041-43ca-94c3-c2bfc9b3215e,le vigne di zamo refosco,1.0,1.0,"{'actual_k': 21, 'was_impossible': False}",167,21,0.0
2,ecc462e1-5041-43ca-94c3-c2bfc9b3215e,le vigne di zamo refosco,1.0,1.0,"{'actual_k': 21, 'was_impossible': False}",167,21,0.0
3,ecc462e1-5041-43ca-94c3-c2bfc9b3215e,Merlot le vigne di zamo,1.0,1.0,"{'actual_k': 23, 'was_impossible': False}",167,23,0.0
4,ecc462e1-5041-43ca-94c3-c2bfc9b3215e,le vigne di zamo refosco,1.0,1.0,"{'actual_k': 21, 'was_impossible': False}",167,21,0.0
5,ecc462e1-5041-43ca-94c3-c2bfc9b3215e,Friulano,1.0,1.2,"{'actual_k': 25, 'was_impossible': False}",167,27,0.2
6,ecc462e1-5041-43ca-94c3-c2bfc9b3215e,Friulano,1.0,1.2,"{'actual_k': 25, 'was_impossible': False}",167,27,0.2
7,ecc462e1-5041-43ca-94c3-c2bfc9b3215e,Merlot le vigne di zamo,1.0,1.0,"{'actual_k': 23, 'was_impossible': False}",167,23,0.0
8,ecc462e1-5041-43ca-94c3-c2bfc9b3215e,Merlot le vigne di zamo,1.0,1.0,"{'actual_k': 23, 'was_impossible': False}",167,23,0.0
9,ecc462e1-5041-43ca-94c3-c2bfc9b3215e,Friulano,1.0,1.2,"{'actual_k': 25, 'was_impossible': False}",167,27,0.2


In [10]:
knn.get_accuracy(predictions)

RMSE: 0.2089
Precision: 0.83
Recall: 0.79


96.48293963254592

* best predictions

In [11]:
best_pred

,uid,iid,rui,est,details,Iu,Ui,err
0,ecc462e1-5041-43ca-94c3-c2bfc9b3215e,Merlot le vigne di zamo,1.0,1.0,"{'actual_k': 23, 'was_impossible': False}",167,23,0.0
1195,3da6eec4-bd99-4370-be23-676baf750f19,Napa Valley Fumé Blanc,4.0,4.0,"{'actual_k': 1, 'was_impossible': False}",7,1,0.0
1194,3da6eec4-bd99-4370-be23-676baf750f19,Monterey Pinot Noir,2.0,2.0,"{'actual_k': 5, 'was_impossible': False}",7,5,0.0
1193,3da6eec4-bd99-4370-be23-676baf750f19,Monterey Pinot Noir,2.0,2.0,"{'actual_k': 5, 'was_impossible': False}",7,5,0.0
1192,3da6eec4-bd99-4370-be23-676baf750f19,Monterey Pinot Noir,2.0,2.0,"{'actual_k': 5, 'was_impossible': False}",7,5,0.0
1191,3da6eec4-bd99-4370-be23-676baf750f19,Monterey Pinot Noir,2.0,2.0,"{'actual_k': 5, 'was_impossible': False}",7,5,0.0
1190,3da6eec4-bd99-4370-be23-676baf750f19,Monterey Pinot Noir,2.0,2.0,"{'actual_k': 5, 'was_impossible': False}",7,5,0.0
1187,25a04c51-5638-42f0-b2e7-ce49b1c280fc,Pigato Riviera Ligure di Ponente,4.0,4.0,"{'actual_k': 2, 'was_impossible': False}",14,2,0.0
1186,25a04c51-5638-42f0-b2e7-ce49b1c280fc,Johannisberger Erntebringer Riesling Feinherb,3.0,3.0,"{'actual_k': 1, 'was_impossible': False}",14,1,0.0
1173,13016d41-00bd-411c-83f5-2b95691696b7,Sauvignon,4.0,4.0,"{'actual_k': 3, 'was_impossible': False}",27,3,0.0


* worst predictions

In [12]:
worst_pred

,uid,iid,rui,est,details,Iu,Ui,err
1169,13016d41-00bd-411c-83f5-2b95691696b7,Pinot grigio,4.0,3.157565,"{'actual_k': 9, 'was_impossible': False}",27,14,0.842435
838,f8c400a6-ded1-40b8-95d2-a263d05c30ce,Barolo,3.0,2.100000,"{'actual_k': 10, 'was_impossible': False}",259,10,0.900000
1744,2ae69fac-d491-412f-8164-ccaf59a230c2,Ronco del Balbo Merlot,4.0,4.900000,"{'actual_k': 10, 'was_impossible': False}",12,10,0.900000
1224,bc8f3005-c2c6-4277-9fd7-340248f4e7ec,Franciacorta,3.0,4.250000,"{'actual_k': 4, 'was_impossible': False}",20,4,1.250000
519,c88644be-2c9b-42f0-9c4b-7f8e81795f43,Bollicina,2.0,3.437837,"{'actual_k': 8, 'was_impossible': False}",27,10,1.437837
520,c88644be-2c9b-42f0-9c4b-7f8e81795f43,Bollicina,2.0,3.437837,"{'actual_k': 8, 'was_impossible': False}",27,10,1.437837
161,ecc462e1-5041-43ca-94c3-c2bfc9b3215e,Cabernet Franc,4.0,2.512312,"{'actual_k': 6, 'was_impossible': False}",167,10,1.487688
555,014e4ed1-6f8b-4b25-917d-c167a2acca17,New Wine,4.0,1.272727,"{'actual_k': 11, 'was_impossible': False}",11,13,2.727273
692,4ed273f7-1816-4f36-88c7-789125f011c7,Moscato d'Asti,5.0,1.285714,"{'actual_k': 28, 'was_impossible': False}",31,35,3.714286
693,4ed273f7-1816-4f36-88c7-789125f011c7,Moscato d'Asti,5.0,1.285714,"{'actual_k': 28, 'was_impossible': False}",31,35,3.714286
